# feature들 합쳐서 분석 가능하게 만들기

### 1. X (IV, control) 다 때려넣기
- 기준 고르기: 원래라면 compu, 걍 app_cit으로 할까? app_cit 한 번이라도 하거나 받은 애들 조사할거니까
- na처리: drop / 0 / 평균 잘 생각

### 2. DV 고르기 (3, 4, 5년 짜리, 공수) 

### 3. dummy 제작 (firm fixed, year fixed)

In [1]:
#-*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np

os.chdir('E:/apps/feature')

### 1. X 만들기
__app cit 기준__

In [2]:
bio = pd.read_csv('appcit_bio.csv')
ee = pd.read_csv('appcit_EE&IT.csv')

len(bio), len(ee)

(1445, 2665)

__patent 붙이기__

In [3]:
df = pd.read_csv('patent_bio.csv')
df1 = pd.read_csv('patent_EE&IT.csv')

bio = bio.merge(df,how='left',on=['gvkey','year'])
ee = ee.merge(df1,how='left',on=['gvkey','year'])

len(bio), len(ee)

(1445, 2665)

In [4]:
bio.isnull().any()

gvkey         False
year          False
app_cit       False
pat_size      False
num_claims    False
bwd           False
fwd           False
self          False
self_fwd      False
dtype: bool

In [5]:
ee.isnull().any()

gvkey         False
year          False
app_cit       False
pat_size      False
num_claims    False
bwd           False
fwd           False
self          False
self_fwd      False
dtype: bool

In [6]:
df = pd.read_csv('patent_self_bio.csv')
df1 = pd.read_csv('patent_self_EE&IT.csv')

df = df.loc[:,['gvkey','year','self_ratio','self_ratio_fwd']]
df1 = df1.loc[:,['gvkey','year','self_ratio','self_ratio_fwd']]

bio = bio.merge(df,how='left',on=['gvkey','year'])
ee = ee.merge(df1,how='left',on=['gvkey','year'])

ee.isnull().any()

gvkey             False
year              False
app_cit           False
pat_size          False
num_claims        False
bwd               False
fwd               False
self              False
self_fwd          False
self_ratio        False
self_ratio_fwd    False
dtype: bool

In [7]:
bio.isnull().any()

gvkey             False
year              False
app_cit           False
pat_size          False
num_claims        False
bwd               False
fwd               False
self              False
self_fwd          False
self_ratio         True
self_ratio_fwd     True
dtype: bool

여기는 얘네 둘 쓰려면 __dropna__ 필요

__compustat 붙이기__

In [8]:
df = pd.read_csv('compu_bio.csv')
df1 = pd.read_csv('compu_EE&IT.csv')

list(df)

['gvkey',
 'fyear',
 'conm',
 'consol',
 'datadate',
 'fic',
 'curcd',
 'act',
 'at',
 'ebit',
 'emp',
 'ppegt',
 'ppent',
 'revt',
 'teq',
 'xlr',
 'xrd',
 'xstfws',
 'ni',
 'sic',
 'roa',
 'margin',
 'rdi',
 'revt_sic4',
 'sic_3',
 'revt_sic3',
 'mkt_4',
 'mkt_3']

In [9]:
df = df.loc[:,['gvkey','fyear','roa','margin','rdi','mkt_4','mkt_3']]
df1 = df1.loc[:,['gvkey','fyear','roa','margin','rdi','mkt_4','mkt_3']]

df = df.rename(columns ={'fyear':'year'})
df1 = df1.rename(columns ={'fyear':'year'})

bio = bio.merge(df,how='left',on=['gvkey','year'])
ee = ee.merge(df1,how='left',on=['gvkey','year'])

ee.isnull().any()

gvkey             False
year              False
app_cit           False
pat_size          False
num_claims        False
bwd               False
fwd               False
self              False
self_fwd          False
self_ratio        False
self_ratio_fwd    False
roa                True
margin             True
rdi                True
mkt_4              True
mkt_3              True
dtype: bool

In [10]:
bio.isnull().any()

gvkey             False
year              False
app_cit           False
pat_size          False
num_claims        False
bwd               False
fwd               False
self              False
self_fwd          False
self_ratio         True
self_ratio_fwd     True
roa                True
margin             True
rdi                True
mkt_4              True
mkt_3              True
dtype: bool

여기는 얘네 둘 쓰려면 __average__ 때려버리자

__tech concentration 붙이기__

In [11]:
df = pd.read_csv('cls_herf_bio.csv')
df1 = pd.read_csv('cls_herf_EE&IT.csv')

bio = bio.merge(df,how='left',on=['gvkey','year'])
ee = ee.merge(df1,how='left',on=['gvkey','year'])

ee.isnull().any()

gvkey             False
year              False
app_cit           False
pat_size          False
num_claims        False
bwd               False
fwd               False
self              False
self_fwd          False
self_ratio        False
self_ratio_fwd    False
roa                True
margin             True
rdi                True
mkt_4              True
mkt_3              True
herf_cls          False
dtype: bool

In [12]:
bio.isnull().any()

gvkey             False
year              False
app_cit           False
pat_size          False
num_claims        False
bwd               False
fwd               False
self              False
self_fwd          False
self_ratio         True
self_ratio_fwd     True
roa                True
margin             True
rdi                True
mkt_4              True
mkt_3              True
herf_cls          False
dtype: bool

In [13]:
bio.to_csv('X_bio_na.csv',index=False)
ee.to_csv('X_EE&IT_na.csv',index=False)

### 2. DV 고르기
- 공수 붙이기: 공수 outer join, X에 left join 후 fillna
- 3, 4, 5년은 따로

In [2]:
bio = pd.read_csv('X_bio_na.csv')
ee = pd.read_csv('X_EE&IT_na.csv')

ba = pd.read_csv('lit_a_3_bio.csv')
bd = pd.read_csv('lit_d_3_bio.csv')

ea = pd.read_csv('lit_a_3_EE&IT.csv')
ed = pd.read_csv('lit_d_3_EE&IT.csv')

print len(ba), len(bd), len(ea), len(ed)

848 872 2032 2320


In [3]:
b = ba.merge(bd,how='outer',on=['gvkey','year'])
e = ea.merge(ed,how='outer',on=['gvkey','year'])

len(b), len(e)

(968, 2512)

In [4]:
len(bio), len(ee)

(1445, 2665)

In [5]:
bio = bio.merge(b,how='left',on=['gvkey','year'])
ee = ee.merge(e,how='left',on=['gvkey','year'])

bio.loc[:, ['lit_a_3', 'lit_d_3']] = bio.loc[:, ['lit_a_3', 'lit_d_3']].fillna(0)
ee.loc[:, ['lit_a_3', 'lit_d_3']] = ee.loc[:, ['lit_a_3', 'lit_d_3']].fillna(0)

ee.isnull().any()

gvkey             False
year              False
app_cit           False
pat_size          False
num_claims        False
bwd               False
fwd               False
self              False
self_fwd          False
self_ratio        False
self_ratio_fwd    False
roa                True
margin             True
rdi                True
mkt_4              True
mkt_3              True
herf_cls          False
lit_a_3           False
lit_d_3           False
dtype: bool

In [6]:
bio.isnull().any()

gvkey             False
year              False
app_cit           False
pat_size          False
num_claims        False
bwd               False
fwd               False
self              False
self_fwd          False
self_ratio         True
self_ratio_fwd     True
roa                True
margin             True
rdi                True
mkt_4              True
mkt_3              True
herf_cls          False
lit_a_3           False
lit_d_3           False
dtype: bool

### 3. dummy들 만들기

In [7]:
bio = pd.get_dummies(bio, dummy_na=True, columns=['gvkey','year'])
ee = pd.get_dummies(ee, dummy_na=True, columns=['gvkey','year'])

bio.to_csv('X_bio_na_dum_3.csv',index=False)
ee.to_csv('X_EE&IT_na_dum_3.csv',index=False)

### 4. 보수공사
- compustat에서 빠진 변수
- patent에서 빠진 변수

__compustat에서 빠진 변수__

In [2]:
bio = pd.read_csv('X_bio_na.csv')
ee = pd.read_csv('X_EE&IT_na.csv')

df = pd.read_csv('compu_bio.csv')
df1 = pd.read_csv('compu_EE&IT.csv')

df = df.loc[:,['gvkey','fyear','xrd','emp']]
df1 = df1.loc[:,['gvkey','fyear','xrd','emp']]

df = df.rename(columns ={'fyear':'year'})
df1 = df1.rename(columns ={'fyear':'year'})

In [3]:
def emp(x):
    if x>0:
        return np.log(x*1000)
    else:
        return np.nan
    
def rnd(x):
    if x>0:
        return np.log(x*1000000)
    else:
        return np.nan

df['emp'] = df['emp'].apply(emp)
df['rnd'] = df['xrd'].apply(rnd)
del df['xrd']

df1['emp'] = df1['emp'].apply(emp)
df1['rnd'] = df1['xrd'].apply(rnd)
del df1['xrd']

bio = bio.merge(df,how='left',on=['gvkey','year'])
ee = ee.merge(df1,how='left',on=['gvkey','year'])

In [4]:
bio.isnull().any()

gvkey             False
year              False
app_cit           False
pat_size          False
num_claims        False
bwd               False
fwd               False
self              False
self_fwd          False
self_ratio         True
self_ratio_fwd     True
roa                True
margin             True
rdi                True
mkt_4              True
mkt_3              True
herf_cls          False
emp                True
rnd                True
dtype: bool

In [5]:
ee.isnull().any()

gvkey             False
year              False
app_cit           False
pat_size          False
num_claims        False
bwd               False
fwd               False
self              False
self_fwd          False
self_ratio        False
self_ratio_fwd    False
roa                True
margin             True
rdi                True
mkt_4              True
mkt_3              True
herf_cls          False
emp                True
rnd                True
dtype: bool

In [6]:
bio.to_csv('X_bio_na1.csv',index=False)
ee.to_csv('X_EE&IT_na1.csv',index=False)

In [7]:
len(bio),len(ee)

(1445, 2665)

In [8]:
bio.dropna(inplace=True)
ee.dropna(inplace=True)

len(bio),len(ee)

(1224, 2157)

In [9]:
bio.to_csv('X_bio_dropna.csv',index=False)
ee.to_csv('X_EE&IT_dropna.csv',index=False)